In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import requests


### Limpeza e tratamento da base
1. Identificando e removendo valores nulos.
2. Convertendo formatos inicialmente desformatados.
3. Removendo outliers.

In [ ]:
# Carregar os dados
df = pd.read_csv('airports-database.csv')

# Limpeza e transformação dos dados
df['dep_time'] = pd.to_datetime(df['dep_time'], format='%H%M', errors='coerce')
df['dep_time'] = df['dep_time'].dt.time
df['sched_dep_time'] = pd.to_datetime(df['sched_dep_time'], format='%H%M', errors='coerce')
df['sched_dep_time'] = df['sched_dep_time'].dt.time
df['arr_time'] = pd.to_datetime(df['arr_time'], format='%H%M', errors='coerce')
df['arr_time'] = df['arr_time'].dt.time
print(f"Número total da base: {df.shape[0]}\n")
nulos_por_coluna = df.isnull().sum()
linhas_com_nulos = df.isnull().any(axis=1).sum()
print(f"Número total de linhas com pelo menos um valor nulo: {linhas_com_nulos}\n")
print(f"Número de nulos por coluna:\n{nulos_por_coluna}\n")
df.dropna(axis=0,how='any',inplace=True)
print(f"Número total da base sem nulos: {df.shape[0]}")

In [4]:
# removendo outliers:
Q1 = df['arr_delay'].quantile(0.25)
Q3 = df['arr_delay'].quantile(0.75)
IQR = Q3 - Q1

# definir limites
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# filtrar os dados
df_filtrado = df[(df['arr_delay'] >= limite_inferior) & (df['arr_delay'] <= limite_superior)]


In [5]:
df = df_filtrado.copy()

### Criando modelo
1. Seleção de features e target.
2. Divisão das bases de treino e de teste (80/20).
3. Treino do modelo.

In [ ]:
# Seleção de features (X) e target (y)
features = ['dep_delay','origin','dest', 'carrier', 'distance']
X = pd.get_dummies(df[features], drop_first=True)
y = df['arr_delay']

# Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=135)

# Treinamento do modelo
model = LinearRegression()
model.fit(X_train, y_train)

### Avaliação do Modelo (métricas):
1. Testando o modelo na base de teste.
2. Métricas do ajuste do target previsto comparado com o target real.

In [7]:
# Avaliação do modelo na base teste
y_pred = model.predict(X_test)

# Calculando RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')

# Calculando MSE
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

# Calculando MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'MAE: {mae}')

# Calculando R²
r2 = r2_score(y_test, y_pred)
print(f'R²: {r2}')

RMSE: 15.412886202963278
MSE: 237.55706110549576
MAE: 11.787583673534222
R²: 0.4652730859605828


C:\Users\vinis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Salvando modelo treinado
1. Gerando o arquivo pickle.
2. Salvando informações relevantes além do modelo para o microserviço dentro do pickle.

In [8]:
import pickle

dummies_columns = X_train.columns
# Salvar o modelo treinado em um arquivo pickle
with open('model.pkl', 'wb') as f:
    pickle.dump({"model": model, "features": features, "dummies_columns": dummies_columns}, f)

### Testes em "prod"
1. Testando predição de um dado aleatório.
2. Carregando o pickle na API no *model/load*.
3. Testando request com payload para *model/predict*.
4. Testando o *model/history*

In [9]:
new_data = pd.DataFrame({
    'dep_delay': [-1, 8],  # Exemplo de valores
    'origin': ['EWR', 'EWR'],
    'dest': ['LAS', 'ORD'],
    'carrier': ['UA', 'MQ'],
    'distance': [2227, 719]
})

# Aplicar a mesma transformação usada no treinamento
new_data_encoded = pd.get_dummies(new_data, drop_first=True)
new_data_encoded = new_data_encoded.reindex(columns=X.columns, fill_value=0)

# Fazer previsões
predictions = model.predict(new_data_encoded)

# Mostrar previsões
for i, pred in enumerate(predictions):
    print(f'Previsão para o novo dado {i+1}: {pred}')

Previsão para o novo dado 1: -22.420311454039954
Previsão para o novo dado 2: -4.302383768867181


In [ ]:
# URL do endpoint
url = "http://127.0.0.1:8080/model/load/"

# Caminho para o arquivo .pkl
file_path = "model.pkl"

# Enviar a solicitação POST com o arquivo
with open(file_path, 'rb') as file:
    response = requests.post(url, files={"file": file})

# Exibir a resposta
print(response.json())

In [ ]:
url = "http://127.0.0.1:8080/model/predict"
data = {
    "dep_delay": 10,
    "origin": "JF",
    "dest": "ORDa",
    "carrier": "AA",
    "distance": 400
}

response = requests.post(url, json=data)
print(response.json())
print(response.status_code)

In [ ]:
url = "http://127.0.0.1:8080/model/history"


response = requests.get(url)
print(response.json())